In [1]:
import duckdb
import leafmap
import pandas as pd

In [2]:
con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")
con.install_extension("spatial")
con.load_extension("spatial")

In [3]:
# check file exists
import os
print(os.path.exists(
    './CurrentSawmill/Current_Wood_Facility_Database_Primary_Wood_Processing.shp'))

True


In [8]:
import os
print(os.path.exists('./test/wash.csv'))

False


In [4]:
# Create sawmill table from shp file and show
con.sql('''
    CREATE TABLE IF NOT EXISTS sawmill as
    SELECT * FROM ST_Read('./CurrentSawmill/Current_Wood_Facility_Database_Primary_Wood_Processing.shp')
''')
con.table('sawmill')

┌───────┬─────────────┬──────────────┬───┬────────────┬──────────────────────┬──────────┬──────────────────────┐
│ RecID │  latitude   │  longitude   │ … │    Year    │      JoinSymbol      │ ObjectId │         geom         │
│ int32 │   double    │    double    │   │    date    │       varchar        │  int32   │       geometry       │
├───────┼─────────────┼──────────────┼───┼────────────┼──────────────────────┼──────────┼──────────────────────┤
│  9001 │ 40.79719014 │ -124.1815646 │ … │ 2020-01-01 │ Operational Large …  │       30 │ POINT (-124.181564…  │
│  9002 │ 38.90448023 │ -121.3070281 │ … │ 2020-01-01 │ Operational Large …  │       31 │ POINT (-121.307028…  │
│  9003 │ 39.48193681 │ -121.5632263 │ … │ 2020-01-01 │ Operational Cedar …  │       32 │ POINT (-121.563226…  │
│  9008 │   40.471498 │  -122.320991 │ … │ 2020-01-01 │ Operational Large …  │       33 │ POINT (-122.320991…  │
│  9012 │   40.901169 │  -124.071143 │ … │ 2020-01-01 │ Operational Fence …  │       34 │ POINT 

In [15]:
# View sawmill table schema
con.sql('''
    DESCRIBE sawmill
        
''')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ RecID       │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ latitude    │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ longitude   │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ Name        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Status      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Owner       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Cogenerati  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Facility_T  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Feedstock_  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Employees   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ StAddr      │ VARCHAR     │ YES     │ NULL    

In [16]:
# Sum sawmills by county
con.sql('''
    SELECT County, COUNT(*) as Count
    FROM sawmill
    GROUP BY County
    ORDER BY count DESC
''')

┌────────────┬───────┐
│   County   │ Count │
│  varchar   │ int64 │
├────────────┼───────┤
│ Shasta     │     5 │
│ Humboldt   │     4 │
│ Sonoma     │     3 │
│ Butte      │     2 │
│ Mendocino  │     2 │
│ Siskiyou   │     2 │
│ Tuolumne   │     2 │
│ Plumas     │     2 │
│ Trinity    │     1 │
│ Lake, OR   │     1 │
│ Placer     │     1 │
│ Tulare     │     1 │
│ Santa Cruz │     1 │
├────────────┴───────┤
│ 13 rows  2 columns │
└────────────────────┘

In [21]:
# Create biomass table from the shp file and show
con.sql('''
    CREATE TABLE IF NOT EXISTS biomass as
    SELECT * FROM ST_Read('./CurrentBiomass/Current_Wood_Facility_Database_Biomass.shp')
''')
con.table('biomass')

┌───────┬─────────────┬──────────────┬──────────────────────┬───┬────────────┬──────────┬──────────────────────┐
│ RecID │  latitude   │  longitude   │         Name         │ … │ JoinSymbol │ ObjectId │         geom         │
│ int32 │   double    │    double    │       varchar        │   │  varchar   │  int32   │       geometry       │
├───────┼─────────────┼──────────────┼──────────────────────┼───┼────────────┼──────────┼──────────────────────┤
│    25 │   35.576448 │  -119.005818 │ DTE Mt. Poso Cogen   │ … │ 25_2021    │       27 │ POINT (-119.005818…  │
│    14 │   36.569581 │  -119.418984 │ Dinuba Energy        │ … │ 14_2021    │       56 │ POINT (-119.418984…  │
│    15 │   38.524353 │  -121.903077 │ Dixon Ridge Farms …  │ … │ 15_2021    │       94 │ POINT (-121.903077…  │
│    16 │   37.943945 │  -121.330053 │ DTE Stockton Bioma…  │ … │ 16_2021    │      110 │ POINT (-121.330053…  │
│    22 │   36.755903 │  -120.365273 │ Covanta Mendota Po…  │ … │ 22_2021    │      122 │ POINT 

In [22]:
# View biomass table schema
con.sql('''
    DESCRIBE biomass
''')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ RecID       │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ latitude    │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ longitude   │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ Name        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Status      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Owner       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Cogenerati  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Facility_T  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Feedstock_  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ MW_Namepla  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ MW_Grid     │ VARCHAR     │ YES     │ NULL    

In [24]:
# Perform spatial join and create new table
con.sql('''
    CREATE TABLE bio_join AS
    SELECT *
    FROM biomass
    JOIN sawmill
    ON biomass.County = sawmill.County
''')

# Verify that the new table has been created
print(con.table('bio_join'))

┌───────┬───────────┬─────────────┬───┬────────────┬──────────────────────┬────────────┬──────────────────────┐
│ RecID │ latitude  │  longitude  │ … │   Year:1   │     JoinSymbol:1     │ ObjectId:1 │        geom:1        │
│ int32 │  double   │   double    │   │    date    │       varchar        │   int32    │       geometry       │
├───────┼───────────┼─────────────┼───┼────────────┼──────────────────────┼────────────┼──────────────────────┤
│    14 │ 36.569581 │ -119.418984 │ … │ 2020-01-01 │ Operational Large …  │         48 │ POINT (-119.048917…  │
│    28 │ 37.874644 │ -120.477546 │ … │ 2020-01-01 │ Operational Large …  │         47 │ POINT (-120.317478…  │
│     5 │ 40.879724 │ -121.720406 │ … │ 2020-01-01 │ Operational Post/P…  │         54 │ POINT (-122.345292…  │
│     6 │ 40.909703 │ -121.639961 │ … │ 2020-01-01 │ Operational Post/P…  │         54 │ POINT (-122.345292…  │
│     7 │ 41.475178 │ -122.829037 │ … │ 2020-01-01 │ Operational Peeler…  │         53 │ POINT (-122.593

In [32]:
result = con.sql('''
    SELECT county, COUNT(*) as count
    FROM bio_join
    GROUP BY County
''')
print(result)

┌───────────┬───────┐
│  County   │ count │
│  varchar  │ int64 │
├───────────┼───────┤
│ Humboldt  │    12 │
│ Tuolumne  │     8 │
│ Shasta    │    40 │
│ Plumas    │     8 │
│ Tulare    │     2 │
│ Placer    │     2 │
│ Siskiyou  │     6 │
│ Butte     │     4 │
│ Mendocino │     4 │
└───────────┴───────┘

